# **Inisialisasi**

In [ ]:
!pip install transformers Sastrawi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 4.6 MB/s eta 0:00:00


In [ ]:
import json
import pandas as pd
import numpy as np
import nltk
from nltk import word_tokenize
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
factory       = StemmerFactory()
stemmer       = factory.create_stemmer()
import random
import pickle
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

# **Data Preperation**


In [ ]:
#Load file json
def load_json_file(filename):
    with open(filename) as f:
        file = json.load(f)
    return file

filename = '/content/intents.json'
intents = load_json_file(filename)

In [ ]:
def create_df(intents):
    patterns = []
    tags = []

    for intent in intents['intents']:
        for pattern in intent['patterns']:
            patterns.append(pattern)
            tags.append(intent['tag'])

    df = pd.DataFrame({
        'patterns': patterns,
        'tag': tags
    })

    return df

df = create_df(intents)
df

,patterns,tag
0,Apa itu gempa bumi?,pengertian_gempa
1,Jelaskan gempa bumi,pengertian_gempa
2,Pengertian dari gempa bumi,pengertian_gempa
3,Gempa bumi itu apa?,pengertian_gempa
4,Pengertian gempa bumi,pengertian_gempa
...,...,...
1049,Bagaimana saya dapat memberikan kontribusi kep...,peran_yang_bisa_dilakukan
1050,Apakah ada cara bagi saya untuk membantu merin...,peran_yang_bisa_dilakukan
1051,Saya ingin memberikan dukungan kepada korban g...,peran_yang_bisa_dilakukan
1052,Adakah peran yang dapat saya ambil untuk mendu...,peran_yang_bisa_dilakukan


In [ ]:
labels = df['tag'].unique().tolist()
labels = [s.strip() for s in labels]
labels

['pengertian_gempa',
 'penyebab_gempa',
 'jenis_gempa',
 'gempa_tektonik',
 'gempa_vulkanik',
 'gempa_dampak_meteorit',
 'gempa_buatan',
 'skala_gempa',
 'alat_ukur_gempa',
 'richter',
 'magnitudo',
 'tingkatan_skala_magnitudo',
 'perbedaan_skala_ritcher_dan_magnitudo',
 'hiposentrum_dan_episentrum',
 'dampak_gempa',
 'seismik',
 'seismologi',
 'P_wave',
 'S_wave',
 'tsunami',
 'tsunami_akibat_gempa',
 'tsunami_terparah_indonesia',
 'bisakah_prediksi_gempa',
 'bisakah_prediksi_tsunami',
 'hewan_prediksi_gempa',
 'wilayah_rawan_gempa',
 'patahan',
 'patahan_naik',
 'patahan_normal',
 'patahan_geser',
 'patahan_divergen',
 'patahan_terbalik',
 'patahan_miring',
 'gempa_akibat_patahan',
 'lempeng',
 'lapisan_bumi',
 'kerak_bumi',
 'jenis_kerak',
 'mantel',
 'inti',
 'mitigasi',
 'persiapan',
 'rencana_tanggap',
 'siap_dokumen',
 'mitigasi_dalam_gedung',
 'mitigasi_dalam_rumah',
 'mitigasi_dalam_lift',
 'mitigasi_luar_ruangan',
 'mitigasi_tempat_ramai',
 'tempat_paling_aman',
 'setelah_gem

In [ ]:
num_labels = len(labels)
id2label = {id:label for id, label in enumerate(labels)}
label2id = {label:id for id, label in enumerate(labels)}

In [ ]:
id2label

{0: 'pengertian_gempa',
 1: 'penyebab_gempa',
 2: 'jenis_gempa',
 3: 'gempa_tektonik',
 4: 'gempa_vulkanik',
 5: 'gempa_dampak_meteorit',
 6: 'gempa_buatan',
 7: 'skala_gempa',
 8: 'alat_ukur_gempa',
 9: 'richter',
 10: 'magnitudo',
 11: 'tingkatan_skala_magnitudo',
 12: 'perbedaan_skala_ritcher_dan_magnitudo',
 13: 'hiposentrum_dan_episentrum',
 14: 'dampak_gempa',
 15: 'seismik',
 16: 'seismologi',
 17: 'P_wave',
 18: 'S_wave',
 19: 'tsunami',
 20: 'tsunami_akibat_gempa',
 21: 'tsunami_terparah_indonesia',
 22: 'bisakah_prediksi_gempa',
 23: 'bisakah_prediksi_tsunami',
 24: 'hewan_prediksi_gempa',
 25: 'wilayah_rawan_gempa',
 26: 'patahan',
 27: 'patahan_naik',
 28: 'patahan_normal',
 29: 'patahan_geser',
 30: 'patahan_divergen',
 31: 'patahan_terbalik',
 32: 'patahan_miring',
 33: 'gempa_akibat_patahan',
 34: 'lempeng',
 35: 'lapisan_bumi',
 36: 'kerak_bumi',
 37: 'jenis_kerak',
 38: 'mantel',
 39: 'inti',
 40: 'mitigasi',
 41: 'persiapan',
 42: 'rencana_tanggap',
 43: 'siap_dokumen

In [ ]:
label2id

{'pengertian_gempa': 0,
 'penyebab_gempa': 1,
 'jenis_gempa': 2,
 'gempa_tektonik': 3,
 'gempa_vulkanik': 4,
 'gempa_dampak_meteorit': 5,
 'gempa_buatan': 6,
 'skala_gempa': 7,
 'alat_ukur_gempa': 8,
 'richter': 9,
 'magnitudo': 10,
 'tingkatan_skala_magnitudo': 11,
 'perbedaan_skala_ritcher_dan_magnitudo': 12,
 'hiposentrum_dan_episentrum': 13,
 'dampak_gempa': 14,
 'seismik': 15,
 'seismologi': 16,
 'P_wave': 17,
 'S_wave': 18,
 'tsunami': 19,
 'tsunami_akibat_gempa': 20,
 'tsunami_terparah_indonesia': 21,
 'bisakah_prediksi_gempa': 22,
 'bisakah_prediksi_tsunami': 23,
 'hewan_prediksi_gempa': 24,
 'wilayah_rawan_gempa': 25,
 'patahan': 26,
 'patahan_naik': 27,
 'patahan_normal': 28,
 'patahan_geser': 29,
 'patahan_divergen': 30,
 'patahan_terbalik': 31,
 'patahan_miring': 32,
 'gempa_akibat_patahan': 33,
 'lempeng': 34,
 'lapisan_bumi': 35,
 'kerak_bumi': 36,
 'jenis_kerak': 37,
 'mantel': 38,
 'inti': 39,
 'mitigasi': 40,
 'persiapan': 41,
 'rencana_tanggap': 42,
 'siap_dokumen': 4

In [ ]:
df['labels'] = df['tag'].map(lambda x: label2id[x.strip()])
df

,patterns,tag,labels
0,Apa itu gempa bumi?,pengertian_gempa,0
1,Jelaskan gempa bumi,pengertian_gempa,0
2,Pengertian dari gempa bumi,pengertian_gempa,0
3,Gempa bumi itu apa?,pengertian_gempa,0
4,Pengertian gempa bumi,pengertian_gempa,0
...,...,...,...
1049,Bagaimana saya dapat memberikan kontribusi kep...,peran_yang_bisa_dilakukan,101
1050,Apakah ada cara bagi saya untuk membantu merin...,peran_yang_bisa_dilakukan,101
1051,Saya ingin memberikan dukungan kepada korban g...,peran_yang_bisa_dilakukan,101
1052,Adakah peran yang dapat saya ambil untuk mendu...,peran_yang_bisa_dilakukan,101


In [ ]:
intents = json.loads(open('/content/intents.json').read())
words = []
classes = []
documents = []
ignore_letters = ['?', '!',',','.']

for intent in intents['intents']:
    for pattern in intent['patterns']:
        word_list = word_tokenize(pattern)
        words.extend(word_list)
        documents.append((word_list,intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

words = [stemmer.stem(word) for word in words if word not in ignore_letters]
words = sorted(set(words))
classes = sorted(set(classes))

pickle.dump(words, open('/content/words.pkl', 'wb'))
pickle.dump(classes, open('/content/classes.pkl', 'wb'))

training = []
output_empty = [0] * len(classes)

for document in documents:
    bag =[]
    word_patterns = document[0]
    word_patterns = [stemmer.stem(word.lower()) for word in word_patterns]
    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)

    output_row = list(output_empty)
    output_row[classes.index(document[1])] = 1
    training.append([bag, output_row])

# Shuffle the training data
random.shuffle(training)
training = np.array(training, dtype=object)

# Create train_x and train_y
train_x = np.array([i[0] for i in training])
train_y = np.array([i[1] for i in training])

# **Model Building**

# **Sequential**

In [ ]:
# Build the model
model = Sequential()
model.add(Input(shape=(len(train_x[0]),)))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile the model
sgd = SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

# Train the model
hist = model.fit(train_x, train_y, epochs=300, batch_size=32, validation_split=0.2, verbose=1)

# Save the model
model.save('chatbotmodel.h5', hist)

print('Done')

Epoch 1/300
27/27 [==============================] - 3s 14ms/step - loss: 4.6232 - accuracy: 0.0119 - val_loss: 4.6287 - val_accuracy: 0.0142
Epoch 2/300
27/27 [==============================] - 0s 4ms/step - loss: 4.6131 - accuracy: 0.0142 - val_loss: 4.6163 - val_accuracy: 0.0237
Epoch 3/300
27/27 [==============================] - 0s 7ms/step - loss: 4.5964 - accuracy: 0.0154 - val_loss: 4.6038 - val_accuracy: 0.0284
Epoch 4/300
27/27 [==============================] - 0s 5ms/step - loss: 4.5790 - accuracy: 0.0344 - val_loss: 4.5908 - val_accuracy: 0.0379
Epoch 5/300
27/27 [==============================] - 0s 5ms/step - loss: 4.5533 - accuracy: 0.0320 - val_loss: 4.5733 - val_accuracy: 0.0664
Epoch 6/300
27/27 [==============================] - 0s 5ms/step - loss: 4.5429 - accuracy: 0.0439 - val_loss: 4.5554 - val_accuracy: 0.0853
Epoch 7/300
27/27 [==============================] - 0s 5ms/step - loss: 4.5008 - accuracy: 0.0486 - val_loss: 4.5327 - val_accuracy: 0.0995
Epoch 8/300


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
# Load the model and data
model = load_model('/content/chatbotmodel.h5')
classes = pickle.load(open('/content/classes.pkl', 'rb'))

# Predict the classes for the training data
predictions = model.predict(train_x)
predicted_classes = np.argmax(predictions, axis=1)
true_classes = np.argmax(train_y, axis=1)

# Classification report
print("Classification Report:")
report = classification_report(true_classes, predicted_classes, target_names=classes)
print(report)

33/33 [==============================] - 0s 2ms/step
Classification Report:
                                       precision    recall  f1-score   support

                               P_wave       1.00      1.00      1.00        18
                               S_wave       1.00      1.00      1.00        17
                           aftershock       1.00      1.00      1.00         9
                      alat_ukur_gempa       1.00      0.92      0.96        13
                 bangunan_tahan_gempa       1.00      1.00      1.00         5
                  bantuan_pasca_gempa       1.00      1.00      1.00         8
                             basarnas       1.00      1.00      1.00        12
               bisakah_prediksi_gempa       1.00      1.00      1.00         9
             bisakah_prediksi_tsunami       1.00      1.00      1.00         9
                                 bmkg       1.00      1.00      1.00        12
                                 bnpb       1.00      

In [ ]:
intents = json.loads(open('/content/intents.json').read())
words = pickle.load(open('/content/words.pkl', 'rb'))
classes = pickle.load(open('/content/classes.pkl', 'rb'))
model = load_model('/content/chatbotmodel.h5')

def clean_up_sentence(sentence):
    sentence_words = word_tokenize(sentence)
    sentence_words = [stemmer.stem(word) for word in sentence_words]
    return sentence_words

def bag_of_words(sentence):
    sentence_words= clean_up_sentence(sentence)
    bag = [0] * len(words)
    for w in sentence_words:
        for i, word in enumerate(words):
            if word == w:
                bag[i] = 1
    print(f"Bag of words: {bag}")  # Print the bag of words array
    return np.array(bag)

def predict_class(sentence):
    bow = bag_of_words(sentence)
    res = model.predict(np.array([bow]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]

    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({'intent': classes[r[0]], 'probability': str(r[1])})
    print(f"Prediction probabilities: {return_list}")
    return return_list


def get_response(intents_list, intents_json):
    if not intents_list:
        return "Maaf, saya tidak mengerti apa yang Anda maksud."
    tag = intents_list[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if i['tag'] == tag:
            return random.choice(i['responses'])
    return "Maaf, saya tidak mengerti apa yang Anda maksud."

def chat():
    print("Hi!!. Saya adalah QuakeBot, assisten virtual yang siap menjawab pertanyaan yang anda miliki mengenai Gempa Bumi. Silahkan bertanya!! ")
    while True:
        message = input("| Kamu: ")
        ints = predict_class(message)
        res = get_response(ints, intents)
        print("| Bot:", res)

        # Check if the predicted intent is "thanks"
        if ints and ints[0]['intent'] == "terima_kasih":
            print("Terima kasih")
            break

In [ ]:
chat()

Hi!!. Saya adalah QuakeBot, assisten virtual yang siap menjawab pertanyaan yang anda miliki mengenai Gempa Bumi. Silahkan bertanya!! 
| Kamu: Pengertian gempa
Bag of words: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0

In [ ]:
chat()


Hi!!. Saya adalah QuakeBot, assisten virtual yang siap menjawab pertanyaan yang anda miliki mengenai Gempa Bumi. Silahkan bertanya!! 
| Kamu: Hubungan sektor agrikultur dengan gempa
1/1 [==============================] - 0s 22ms/step
Prediction probabilities: [{'intent': 'sektor_agrikultur_gempa', 'probability': '0.9946741'}]
| Bot: Gempa bumi dapat menyebabkan kerusakan signifikan pada infrastruktur dan sarana pertanian, seperti penggilingan padi, irigasi, dan lain-lain, yang dapat menghambat proses produksi pertanian. Selain itu, gempa bumi juga dapat menyebabkan trauma pada petani, sehingga mereka tidak dapat menggarap lahan mereka, mengganggu rantai pasok dan harga komoditas pangan yang naik. Gempa bumi juga dapat mempengaruhi produktivitas dan kualitas ternak, serta mengganggu pengelolaan sapi potong model integrasi padi-ternak yang sudah berjalan dengan baik
| Kamu: terima kasih
1/1 [==============================] - 0s 21ms/step
Prediction probabilities: [{'intent': 'terima_kasi

# **S**